In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rc('font',family='Times New Roman')

In [2]:
from sklearn import linear_model, kernel_ridge, svm, gaussian_process, tree
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, PairwiseKernel
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from lightgbm import LGBMRegressor
import joblib

In [3]:
# 创建一个模型配置类
class Config:
    def __init__(self, data_path='./Data/PRSA_Data/Data_Aotizhongxin.csv', timestep=1, train_size=0.75, model_name='model'):
        self.data_path = data_path
        self.timestep = timestep
        self.train_size = train_size
        self.model_name = model_name
        
    def modify_params(self, param, value):
        if param == 'data_path':
            self.data_path = value
        elif param == 'timestep':
            self.timestep = value
        elif param == 'train_size':
            self.train_size = value
        elif param == 'model_name':
            self.model_name = value
        else:
            raise SystemExit('The param is out the range OR value type is false')

In [4]:
# 加载数据并标准化（归一化）
# 数据标准化（归一化）
# 标准化：对原始数据进行变换把数据变换到均值为0,方差为1范围内
# 归一化：对每个特征缩放到给定范围(默认[0,1])
# 一般只对自变量X进行标准化（归一化）
def read_data(data_path):
    df_data = pd.read_csv(data_path, index_col=0)
    # 采用IQR异常值检验和线性插值
    df = df_data.copy()
    df = IQR(df_data, df)
    df = df.interpolate()
    scaler = StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(np.array(df)))
    df.reset_index(inplace=True)
    data = pd.concat([data.iloc[:,1:], df.iloc[:,1]], axis=1,join='outer')
    data = np.array(data)
    return data

def IQR(data, new_data):
    for i in range(len(np.array(data.columns))):
        df_25 = data.iloc[:,i].quantile(0.25)
        df_75 = data.iloc[:,i].quantile(0.75)
        IQR = df_75 - df_25
        lower_limit = df_25 - 1.5*IQR
        upper_limit = df_75 + 1.5*IQR
        new_data.iloc[:,i] = data.iloc[:,i][(data.iloc[:,i]>lower_limit) & (data.iloc[:,i]<upper_limit)]
        return new_data

In [11]:
# 划分训练集、测试集
def split_data(data, timestep, train_size):
    data_X = []
    data_Y = []
    
    # 将整个窗口的数据保存到X中，将未来timestep保存到Y中
    for index in range(len(data)- timestep):
        data_X.append(data[index: index + timestep])
        data_Y.append(data[index + timestep][-1])
    
    dataX = np.array(data_X)
    dataY = np.array(data_Y)
    
    train_size = int(np.round(train_size * dataX.shape[0]))
    
    x_train = dataX[: train_size, :].reshape(-1, timestep, data.shape[1])
    y_train = dataY[: train_size].reshape(-1, 1)
    
    x_test = dataX[train_size:, :].reshape(-1, timestep, data.shape[1])
    y_test = dataY[train_size:].reshape(-1, 1)
    
    x_train = x_train.reshape(x_train.shape[0], (x_train.shape[1] * x_train.shape[2]))
    x_test = x_test.reshape(x_test.shape[0], (x_test.shape[1] * x_test.shape[2]))
    
    x_train = pd.DataFrame(x_train)
    x_test = pd.DataFrame(x_test)
        
    return [x_train, y_train, x_test, y_test]

In [12]:
# 利用各种机器学习算法进行模型训练

In [13]:
# 利用训练并保存好的模型进行预测
def prediction(x_test, y_test, config):
    save_path = './Model/{}.pkl'.format(config.model_name)
    model = joblib.load(save_path)
    y_pre = model.predict(x_test)
    NSE =r2_score(y_test,y_pre)
    MAE =mean_absolute_error(y_test,y_pre)
    return y_pre, NSE, MAE

In [14]:
# 保存预测结果
def save_results(y_test, y_pre, config):
    file_path = './Results/{}.csv'.format(config.model_name)
    df = pd.read_csv(config.data_path, index_col=0)
    train_size = int(np.round(config.train_size * df.shape[0]))
    index = df.index.values[train_size:]
    index = pd.to_datetime(pd.DataFrame(index).iloc[:,0]).values
    result = pd.DataFrame({'datetime': index, 'Measured values': y_test, 'Predicted values':y_pre})
    result.to_csv(file_path, index=False, sep=',')

#### 1.贝叶斯岭回归（Bayesian Ridge Regression）

In [15]:
def Bayesian_regression(x_train, y_train, config):
    reg = linear_model.BayesianRidge()
    param_grid = {"n_iter":[300, 500], "tol":[1e-6], "alpha_1": [1e-5, 1e-6, 1e-7], "alpha_2": [1e-5, 1e-6, 1e-7], "lambda_1": [1e-5, 1e-6, 1e-7], "lambda_2": [1e-5, 1e-6, 1e-7], "fit_intercept": [True, False], "verbose": [True]}
    reg = GridSearchCV(reg, param_grid=param_grid)
    reg.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(reg, save_path)

In [16]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'reg')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
Bayesian_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), y_pre, config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergence after  7  iterations
Convergenc

#### 2.内核岭回归（Kernel ridge regression）

In [11]:
# 训练时间过长，不建议使用网格搜索
def Kernel_ridge(x_train, y_train, config):
    krr = kernel_ridge.KernelRidge(alpha=0.6, kernel='polynomial', degree=3, coef0=1)
    # param_grid = {"alpha":[i for i in np.arange(0, 1, 0.2)], "kernel":['additive_chi2','linear', 'polynomial', 'rbf', 'sigmoid', 'cosine'], "degree":[3], "coef0":[i for i in np.arange(1, 3, 0.5)]}
    # krr = GridSearchCV(krr, param_grid=param_grid)
    krr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(krr, save_path)

In [12]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'krr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
Kernel_ridge(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

NSE= 0.9434020349949396
MAE= 9.04878926297424


#### 3.支持向量机（support vector machines）

In [13]:
def SupportVector_regression(x_train, y_train, config):
    svr = svm.SVR(tol=1e-6, cache_size=200,degree=3,kernel='rbf',coef0=0, C=1,epsilon=0.1, verbose=True)
    param_grid = {"kernel":['poly', 'rbf', 'sigmoid']}
    svr = GridSearchCV(svr, param_grid=param_grid)
    svr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(svr, save_path)

In [14]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'svr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
SupportVector_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print('')
print("NSE=", NSE)
print("MAE=", MAE)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]
NSE= 0.9440721421530993
MAE= 8.792993037895055


#### 4.高斯过程回归

In [17]:
def Gaussian_regression(x_train, y_train, config):
    # kernel=DotProduct() + WhiteKernel()
    kernel = RBF()
    # kernel = PairwiseKernel()
    gpr = gaussian_process.GaussianProcessRegressor(kernel=kernel)
    gpr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(gpr, save_path)
   

# 利用训练并保存好的模型进行预测
def Gaussian_prediction(x_test, y_test, config):
    save_path = './Model/{}.pkl'.format(config.model_name)
    model = joblib.load(save_path)
    y_pre, y_pre_std = model.predict(x_test, return_std=True)
    NSE =r2_score(y_test,y_pre)
    MAE =mean_absolute_error(y_test,y_pre)
    return y_pre, y_pre_std, NSE, MAE

# 保存预测结果
def Gaussian_save_results(y_test, y_pre, y_pre_std, config):
    file_path = './Results/{}.csv'.format(config.model_name)
    df = pd.read_csv(config.data_path, index_col=0)
    train_size = int(np.round(config.train_size * df.shape[0]))
    index = df.index.values[train_size:]
    index = pd.to_datetime(pd.DataFrame(index).iloc[:,0]).values
    result = pd.DataFrame({'datetime': index, 'Measured values': y_test, 'Predicted values':y_pre, 'Predicted std':y_pre_std})
    result.to_csv(file_path, index=False, sep=',')

In [ ]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'gpr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
Gaussian_regression(x_train, y_train, config)
# 6.模型预测
y_pre, y_pre_std, NSE, MAE = Gaussian_prediction(x_test, y_test, config)
# 7.结果保存
Gaussian_save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), np.reshape(y_pre_std, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

#### 5.决策树（Decision Tree）

In [20]:
def DesicionTree_regression(x_train, y_train, config):
    dtr = tree.DecisionTreeRegressor()
    param_grid = {"max_depth":[3, 5, 8, 10, 15]}
    dtr = GridSearchCV(dtr, param_grid=param_grid)
    dtr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(dtr, save_path)

In [21]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'dtr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
DesicionTree_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

NSE= 0.9377546139226798
MAE= 9.38483810654378


#### 6.随机森林（Random Forest）

In [22]:
def Randomforest_regression(x_train, y_train, config):
    rf = RandomForestRegressor(verbose=True, n_jobs=4)
    param_grid = {"n_estimators":[100, 300, 600, 1000], "max_depth":[5, 8, 10], "learning_rate":[0.05, 0.1, 0.5, 1]}
    rf = GridSearchCV(rf, param_grid=param_grid)
    rf.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(rf, save_path)

In [23]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'rf')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
Randomforest_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    5.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[P

NSE= 0.9452051057605388
MAE= 8.853629370155588


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.0s finished


#### 7.AdaBoost

In [26]:
def AdaBoost_regression(x_train, y_train, config):
    abr = AdaBoostRegressor()
    param_grid = {"n_estimators": [100, 300, 600, 1000], "learning_rate":[0.05, 0.1, 0.5, 1]}
    abr = GridSearchCV(abr, param_grid=param_grid)
    abr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(abr, save_path)

In [27]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'abr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
AdaBoost_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

NSE= 0.9301847991194478
MAE= 11.352252341675836


#### 8.Gradient Boosting Decision Tree（GBDT）

In [28]:
def GBDT_regression(x_train, y_train, config):
    gbr = GradientBoostingRegressor(verbose=True)
    param_grid = {"n_estimators": [100, 300, 600, 1000], "learning_rate":[0.05, 0.1, 0.5, 1], "alpha": [0.5, 0.9]}
    gbr = GridSearchCV(gbr, param_grid=param_grid)
    gbr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(gbr, save_path)

In [ ]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'gbr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
GBDT_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

#### 9.XGBoost

In [38]:
def XGBoost_regression(x_train, y_train, config):
    xgbr = XGBRegressor()
    param_grid = {"n_estimators": [100, 300, 600, 1000], "learning_rate":[0.05, 0.1, 0.5, 1], "max_depth": [2, 3, -1]}
    xgbr = GridSearchCV(xgbr, param_grid=param_grid)
    xgbr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(xgbr, save_path)

In [39]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'xgbr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
XGBoost_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

NSE= 0.9471688303244445
MAE= 8.812276209714334


#### 10.Lightgbm

In [40]:
def Lightgbm_regression(x_train, y_train, config):
    gbmr = LGBMRegressor()
    param_grid = {"num_leaves": [21, 31], "max_depth": [1, 3, -1], "learning_rate": [0.01, 0.05, 0.1], "n_estimators": [100, 300, 600, 1000]}
    gbmr = GridSearchCV(gbmr, param_grid=param_grid)
    gbmr.fit(x_train, y_train)
    save_path = './Model/{}.pkl'.format(config.model_name)
    joblib.dump(gbmr, save_path)

In [41]:
# 模型调用过程
# 1.初始化配置
config = Config(train_size=0.8)
# 2.根据使用模型修改名称
config.modify_params('model_name', 'gbmr')
# 3.读取数据
data = read_data(config.data_path)
# 4.划分训练集、测试集
x_train, y_train, x_test, y_test = split_data(data, config.timestep, config.train_size)
# 5.模型训练
Lightgbm_regression(x_train, y_train, config)
# 6.模型预测
y_pre, NSE, MAE = prediction(x_test, y_test, config)
# 7.结果保存
save_results(np.reshape(y_test, -1), np.reshape(y_pre, -1), config)
# 8.绘图展示（4.中展示）
print("NSE=", NSE)
print("MAE=", MAE)

NSE= 0.9477536913666604
MAE= 8.78424760082124
